# 문제 2: 핀다 홈화면 진입 고객의 모델 기반 고객 군집 분석/군집 별 서비스 메시지 제안

user_id 별 1행만 사용(가장 최근 행)
\
5월까지의 데이터만 사용

\
(1) **log_data** : 행동 로그 분석을 위해 전처리한 train_log_data_1.csv 사용
- app_count : 앱 사용빈도
- GetCreditInfo : 신용조회 빈도
- UseLoanManage : 대출관리 빈도
- UsePrepayCalc : 여윳돈 계산기
- UseDSRCalc : DSR 계산기 빈도
- 빈도 별 변동성(variation) : 3, 4, 5월 빈도의 회귀계수
- 신규 유저(new) : 주어진 기간 내 SignUp 이력이 있는 유저

\
(2) **user_spec + loan_result** : 예측 모델링을 위해 전처리한 merged_df.csv 사용
- 핀다 내 대출신청 횟수(applied_cnt) : is_applied==1 총합
- 최근성(recency) : 최근 접속일로부터 6/1까지의 기간


# 데이터 전처리

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cls_df = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/제출파일/Raw 데이터/train_log_data_1.csv')

In [ ]:
cls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13467424 entries, 0 to 13467423
Data columns (total 12 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        int64 
 1   date_cd        object
 2   event          object
 3   timestamp      object
 4   refer_event_1  object
 5   refer_event_2  object
 6   refer_event_3  object
 7   refer_event_4  object
 8   refer_event_5  object
 9   stay_time      int64 
 10  week_pop       int64 
 11  spend_time     int64 
dtypes: int64(4), object(8)
memory usage: 1.2+ GB


In [ ]:
cls_df.drop(['refer_event_2','refer_event_3','refer_event_4','refer_event_5',
             'week_pop','spend_time','stay_time'], axis=1, inplace=True)
cls_df['timestamp'] = pd.to_datetime(cls_df['timestamp'])
cls_df['month'] = cls_df['timestamp'].dt.month

In [ ]:
cls_df

,user_id,date_cd,event,timestamp,refer_event_1,month
0,1,2022-05-03,GetCreditInfo,2022-05-03 14:52:28,start,5
1,1,2022-05-03,GetCreditInfo,2022-05-03 14:52:35,GetCreditInfo,5
2,7,2022-05-22,GetCreditInfo,2022-05-22 16:39:49,start,5
3,9,2022-05-21,GetCreditInfo,2022-05-21 23:37:58,start,5
4,9,2022-05-21,GetCreditInfo,2022-05-21 23:43:33,GetCreditInfo,5
...,...,...,...,...,...,...
13467419,879696,2022-03-14,ClickApply,2022-03-14 05:40:56,CompleteIDCertification,3
13467420,879696,2022-03-14,EndLoanApply,2022-03-14 05:42:14,ClickApply,3
13467421,879698,2022-05-24,OpenApp,2022-05-24 22:33:24,start,5
13467422,879698,2022-05-24,ViewLoanApplyIntro,2022-05-24 22:33:32,OpenApp,5


In [ ]:
print('log_data에서 user_id의 개수: ', cls_df['user_id'].nunique()) #489023

log_data에서 user_id의 개수:  489023


## 앱 사용빈도

앱 사용 로그 중 OpenApp이 있으나 OpenApp 로그 없이도 앱 사용 이력이 있는 로그가 존재. 
\
전처리된 log_data에는 모든 접속 시작 이력이 표시되어 있으므로 이를 사용. (train_log_data_1.csv)

In [ ]:
openapp_cnt = cls_df[cls_df['refer_event_1']=='start'].groupby(['user_id','month']).count()['refer_event_1']
openapp = openapp_cnt.reset_index()
openapp = openapp.rename(columns = {'refer_event_1':'app_count'})
openapp # ['user_id', 'month']개수 795325개 맞음

,user_id,month,app_count
0,1,5,1
1,7,5,1
2,9,5,1
3,11,3,1
4,11,4,1
...,...,...,...
795320,879693,5,5
795321,879694,3,1
795322,879695,5,1
795323,879696,3,1


In [ ]:
# 3, 4, 5월 별 횟수 
openapp['app_count_3'] = openapp['app_count']
openapp.loc[openapp['month']!=3, 'app_count_3'] = 0

openapp['app_count_4'] = openapp['app_count']
openapp.loc[openapp['month']!=4, 'app_count_4'] = 0

openapp['app_count_5'] = openapp['app_count']
openapp.loc[openapp['month']!=5, 'app_count_5'] = 0

In [ ]:
openapp = openapp.groupby('user_id').sum().reset_index().drop('month',axis=1)
openapp # log_data에서 user_id개수 489023개 맞음

,user_id,app_count,app_count_3,app_count_4,app_count_5
0,1,1,0,0,1
1,7,1,0,0,1
2,9,1,0,0,1
3,11,2,1,1,0
4,12,10,2,2,6
...,...,...,...,...,...
489018,879693,5,0,0,5
489019,879694,1,1,0,0
489020,879695,1,0,0,1
489021,879696,1,1,0,0


## 서비스별 이용 빈도

이벤트 별로 One-hot encoding을 하고, user_id별로 총합을 구하여 서비스별 이용 횟수의 총합을 구함.

### encoding

In [ ]:
#event
cls_df['event'].value_counts()

OpenApp                    2477195
EndLoanApply               1994803
GetCreditInfo              1819745
Login                      1780501
StartLoanApply             1399390
ViewLoanApplyIntro         1318472
UseLoanManage              1173186
CompleteIDCertification     907748
ClickApply                  560096
SignUp                       24552
UsePrepayCalc                 7196
UseDSRCalc                    4540
Name: event, dtype: int64

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
event_cat = ohe.fit_transform(cls_df[['event']])

event_cat = pd.DataFrame(event_cat, columns=[col for col in ohe.categories_[0]])

In [ ]:
event_cat[['user_id','month']] = cls_df[['user_id','month']]
event_cat

,ClickApply,CompleteIDCertification,EndLoanApply,GetCreditInfo,Login,OpenApp,SignUp,StartLoanApply,UseDSRCalc,UseLoanManage,UsePrepayCalc,ViewLoanApplyIntro,user_id,month
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,5
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,5
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13467419,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,879696,3
13467420,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,879696,3
13467421,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,879698,5
13467422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,879698,5


### 서비스별 이용 빈도 계산

In [ ]:
event_cnt = event_cat.groupby(['user_id','month']).sum()[['GetCreditInfo','SignUp',
                                                'UseLoanManage','UseDSRCalc','UsePrepayCalc']].reset_index()
event_cnt # ['user_id', 'month']개수 795325개 맞음

,user_id,month,GetCreditInfo,SignUp,UseLoanManage,UseDSRCalc,UsePrepayCalc
0,1,5,2.0,0.0,0.0,0.0,0.0
1,7,5,1.0,0.0,0.0,0.0,0.0
2,9,5,3.0,0.0,0.0,0.0,0.0
3,11,3,3.0,0.0,3.0,0.0,1.0
4,11,4,5.0,0.0,6.0,0.0,0.0
...,...,...,...,...,...,...,...
795320,879693,5,1.0,0.0,3.0,0.0,0.0
795321,879694,3,0.0,0.0,0.0,0.0,0.0
795322,879695,5,0.0,0.0,0.0,0.0,0.0
795323,879696,3,3.0,0.0,2.0,0.0,0.0


In [ ]:
# 3, 4, 5월 별 횟수
event_cnt['credit_count_3'] = event_cnt['GetCreditInfo']
event_cnt.loc[event_cnt['month']!=3, 'credit_count_3'] = 0

event_cnt['credit_count_4'] = event_cnt['GetCreditInfo']
event_cnt.loc[event_cnt['month']!=4, 'credit_count_4'] = 0

event_cnt['credit_count_5'] = event_cnt['GetCreditInfo']
event_cnt.loc[event_cnt['month']!=5, 'credit_count_5'] = 0

In [ ]:
# 3, 4, 5월 별 횟수
event_cnt['loan_count_3'] = event_cnt['UseLoanManage']
event_cnt.loc[event_cnt['month']!=3, 'loan_count_3'] = 0

event_cnt['loan_count_4'] = event_cnt['UseLoanManage']
event_cnt.loc[event_cnt['month']!=4, 'loan_count_4'] = 0

event_cnt['loan_count_5'] = event_cnt['UseLoanManage']
event_cnt.loc[event_cnt['month']!=5, 'loan_count_5'] = 0

In [ ]:
# 3, 4, 5월 별 횟수
event_cnt['dsr_count_3'] = event_cnt['UseDSRCalc']
event_cnt.loc[event_cnt['month']!=3, 'dsr_count_3'] = 0

event_cnt['dsr_count_4'] = event_cnt['UseDSRCalc']
event_cnt.loc[event_cnt['month']!=4, 'dsr_count_4'] = 0

event_cnt['dsr_count_5'] = event_cnt['UseDSRCalc']
event_cnt.loc[event_cnt['month']!=5, 'dsr_count_5'] = 0

In [ ]:
# 3, 4, 5월 별 횟수
event_cnt['prepay_count_3'] = event_cnt['UsePrepayCalc']
event_cnt.loc[event_cnt['month']!=3, 'prepay_count_3'] = 0

event_cnt['prepay_count_4'] = event_cnt['UsePrepayCalc']
event_cnt.loc[event_cnt['month']!=4, 'prepay_count_4'] = 0

event_cnt['prepay_count_5'] = event_cnt['UsePrepayCalc']
event_cnt.loc[event_cnt['month']!=5, 'prepay_count_5'] = 0

In [ ]:
event_cnt = event_cnt.groupby('user_id').sum().reset_index().drop('month',axis=1)
event_cnt # log_data에서 user_id개수 489023개 맞음

,user_id,GetCreditInfo,SignUp,UseLoanManage,UseDSRCalc,UsePrepayCalc,credit_count_3,credit_count_4,credit_count_5,loan_count_3,loan_count_4,loan_count_5,dsr_count_3,dsr_count_4,dsr_count_5,prepay_count_3,prepay_count_4,prepay_count_5
0,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11,8.0,0.0,9.0,0.0,1.0,3.0,5.0,0.0,3.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,12,3.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489018,879693,1.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
489019,879694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489020,879695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489021,879696,3.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 핀다 내 대출횟수

user_spec과 loan_result를 결합한 테이블(merged_df.csv)에서 is_applied=1인 횟수를 구하여 유저가 핀다 내에서 얼마나 대출을 신청했는지 횟수를 구함.

In [3]:
# user_spec과 loan_result가 결합된 테이블 불러오기
result = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/제출파일/Raw 데이터/merged_df.csv') 

In [ ]:
print('user_spec+loan_result에서 user_id의 개수: ', result['user_id'].nunique()) #259313

user_spec+loan_result에서 user_id의 개수:  259313


In [ ]:
applied_cnt = result.groupby(['user_id','month']).sum()['is_applied'].reset_index()

In [ ]:
# 3, 4, 5월 횟수
applied_cnt['applied_count_3'] = applied_cnt['is_applied']
applied_cnt.loc[applied_cnt['month']!=3, 'applied_count_3'] = 0

applied_cnt['applied_count_4'] = applied_cnt['is_applied']
applied_cnt.loc[applied_cnt['month']!=4, 'applied_count_4'] = 0

applied_cnt['applied_count_5'] = applied_cnt['is_applied']
applied_cnt.loc[applied_cnt['month']!=5, 'applied_count_5'] = 0

In [ ]:
applied_cnt = applied_cnt.groupby('user_id').sum().reset_index().drop('month',axis=1)
applied_cnt.rename(columns={'is_applied':'applied_cnt'}, inplace=True)
applied_cnt # user_id개수 259313 맞음

,user_id,applied_cnt,applied_count_3,applied_count_4,applied_count_5
0,9.0,0.0,0.0,0.0,0.0
1,11.0,3.0,2.0,1.0,0.0
2,14.0,0.0,0.0,0.0,0.0
3,17.0,2.0,2.0,0.0,0.0
4,19.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
259308,879692.0,6.0,0.0,6.0,0.0
259309,879693.0,2.0,0.0,0.0,2.0
259310,879695.0,1.0,0.0,0.0,1.0
259311,879696.0,3.0,3.0,0.0,0.0


## 데이터 합치기

In [ ]:
cls_df = cls_df.drop_duplicates(['user_id'], keep='last')
cls_df

,user_id,date_cd,event,timestamp,refer_event_1,month
1,1,2022-05-03,GetCreditInfo,2022-05-03 14:52:35,GetCreditInfo,5
2,7,2022-05-22,GetCreditInfo,2022-05-22 16:39:49,start,5
5,9,2022-05-21,GetCreditInfo,2022-05-21 23:43:52,GetCreditInfo,5
41,11,2022-04-20,UseLoanManage,2022-04-20 10:40:12,GetCreditInfo,4
127,12,2022-05-19,EndLoanApply,2022-05-19 06:27:20,EndLoanApply,5
...,...,...,...,...,...,...
13467395,879693,2022-05-20,EndLoanApply,2022-05-20 16:39:54,StartLoanApply,5
13467401,879694,2022-03-31,EndLoanApply,2022-03-31 20:18:18,EndLoanApply,3
13467406,879695,2022-05-27,EndLoanApply,2022-05-27 12:51:23,ClickApply,5
13467420,879696,2022-03-14,EndLoanApply,2022-03-14 05:42:14,ClickApply,3


In [ ]:
print('log_data와 user+loan_result에 공통된 user_id:',
    len(sorted(list(set(cls_df['user_id']) & set(applied_cnt['user_id'])))))

log_data와 user+loan_result에 공통된 user_id: 246285


In [ ]:
cls_df = pd.merge(cls_df, openapp, how='inner', on='user_id') # log_data에서 만든 데이터라서 총 489023개

In [ ]:
cls_df = pd.merge(cls_df, event_cnt, how='inner', on='user_id') # log_data에서 만든 데이터라서 총 489023개

In [ ]:
cls_df = pd.merge(cls_df, applied_cnt, how='inner', on='user_id') # user+loan_result까지 합해진 데이터라서 총 246285개

## recency

6/1 기준 마지막 접속일로부터 며칠이 지났는지.

In [ ]:
from datetime import datetime

cls_df['recency'] = datetime.strptime('2022-06-01', '%Y-%m-%d') - cls_df['timestamp']
cls_df['recency'] = cls_df['recency'].dt.days

cls_df # user+loan_result까지 합해진 데이터라서 총 246285개

,user_id,date_cd,event,timestamp,refer_event_1,month,app_count,app_count_3,app_count_4,app_count_5,...,dsr_count_4,dsr_count_5,prepay_count_3,prepay_count_4,prepay_count_5,applied_cnt,applied_count_3,applied_count_4,applied_count_5,recency
0,9,2022-05-21,GetCreditInfo,2022-05-21 23:43:52,GetCreditInfo,5,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
1,11,2022-04-20,UseLoanManage,2022-04-20 10:40:12,GetCreditInfo,4,2,1,1,0,...,0.0,0.0,1.0,0.0,0.0,3.0,2.0,1.0,0.0,41
2,17,2022-03-07,EndLoanApply,2022-03-07 13:55:02,StartLoanApply,3,4,4,0,0,...,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,85
3,19,2022-05-14,EndLoanApply,2022-05-14 00:56:59,EndLoanApply,5,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17
4,20,2022-05-23,OpenApp,2022-05-23 01:33:32,start,5,13,10,1,2,...,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246280,879691,2022-03-14,Login,2022-03-14 09:05:24,OpenApp,3,10,10,0,0,...,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,78
246281,879692,2022-04-25,EndLoanApply,2022-04-25 14:00:41,StartLoanApply,4,8,0,8,0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0,0.0,36
246282,879693,2022-05-20,EndLoanApply,2022-05-20 16:39:54,StartLoanApply,5,5,0,0,5,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,11
246283,879695,2022-05-27,EndLoanApply,2022-05-27 12:51:23,ClickApply,5,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4


## 신규유저

주어진 기간(3, 4, 5월) 내 SignUp 이력이 있는 경우.

In [ ]:
cls_df['new'] = 0

cls_df.loc[cls_df['SignUp']>0, 'new'] = 1 # 총 9889명이 신규유저
cls_df

,user_id,date_cd,event,timestamp,refer_event_1,month,app_count,app_count_3,app_count_4,app_count_5,...,dsr_count_5,prepay_count_3,prepay_count_4,prepay_count_5,applied_cnt,applied_count_3,applied_count_4,applied_count_5,recency,new
0,9,2022-05-21,GetCreditInfo,2022-05-21 23:43:52,GetCreditInfo,5,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0
1,11,2022-04-20,UseLoanManage,2022-04-20 10:40:12,GetCreditInfo,4,2,1,1,0,...,0.0,1.0,0.0,0.0,3.0,2.0,1.0,0.0,41,0
2,17,2022-03-07,EndLoanApply,2022-03-07 13:55:02,StartLoanApply,3,4,4,0,0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,85,0
3,19,2022-05-14,EndLoanApply,2022-05-14 00:56:59,EndLoanApply,5,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17,0
4,20,2022-05-23,OpenApp,2022-05-23 01:33:32,start,5,13,10,1,2,...,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246280,879691,2022-03-14,Login,2022-03-14 09:05:24,OpenApp,3,10,10,0,0,...,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,78,0
246281,879692,2022-04-25,EndLoanApply,2022-04-25 14:00:41,StartLoanApply,4,8,0,8,0,...,0.0,0.0,0.0,0.0,6.0,0.0,6.0,0.0,36,0
246282,879693,2022-05-20,EndLoanApply,2022-05-20 16:39:54,StartLoanApply,5,5,0,0,5,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,11,0
246283,879695,2022-05-27,EndLoanApply,2022-05-27 12:51:23,ClickApply,5,1,0,0,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4,0


## 기능별 시계열 변수

In [ ]:
# 회귀계수
b_app_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,8],cls_df.iloc[i,9], cls_df.iloc[i,10]]
  fit_line = np.polyfit(x, y, 1)
  b_app_array.append(fit_line[0])

b_credit_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,16],cls_df.iloc[i,17], cls_df.iloc[i,18]]
  fit_line = np.polyfit(x, y, 1)
  b_credit_array.append(fit_line[0])

b_prepay_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,25],cls_df.iloc[i,26], cls_df.iloc[i,27]]
  fit_line = np.polyfit(x, y, 1)
  b_prepay_array.append(fit_line[0])

b_dsr_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,22],cls_df.iloc[i,23], cls_df.iloc[i,24]]
  fit_line = np.polyfit(x, y, 1)
  b_dsr_array.append(fit_line[0])

b_loan_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,19],cls_df.iloc[i,20], cls_df.iloc[i,21]]
  fit_line = np.polyfit(x, y, 1)
  b_loan_array.append(fit_line[0])

b_applied_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,29],cls_df.iloc[i,30], cls_df.iloc[i,31]]
  fit_line = np.polyfit(x, y, 1)
  b_applied_array.append(fit_line[0])

In [ ]:
cls_df['app_variation'] = b_app_array
cls_df['credit_variation'] = b_credit_array
cls_df['prepay_variation'] = b_prepay_array
cls_df['dsr_variation'] = b_dsr_array
cls_df['loan_variation'] = b_loan_array
cls_df['applied_variation'] = b_applied_array

In [ ]:
cls_df

,user_id,date_cd,event,timestamp,refer_event_1,month,app_count,app_count_3,app_count_4,app_count_5,...,applied_count_4,applied_count_5,recency,new,app_variation,credit_variation,prepay_variation,dsr_variation,loan_variation,applied_variation
0,9,2022-05-21,GetCreditInfo,2022-05-21 23:43:52,GetCreditInfo,5,1,0,0,1,...,0.0,0.0,10,0,1.500000e+00,1.583361e-15,0.0,0.0,0.000000e+00,5.0
1,11,2022-04-20,UseLoanManage,2022-04-20 10:40:12,GetCreditInfo,4,2,1,1,0,...,1.0,0.0,41,0,3.500000e+00,-1.000000e+00,1.5,0.5,-3.000000e+00,20.0
2,17,2022-03-07,EndLoanApply,2022-03-07 13:55:02,StartLoanApply,3,4,4,0,0,...,0.0,0.0,85,0,1.000000e+00,5.000000e-01,1.0,0.0,0.000000e+00,42.5
3,19,2022-05-14,EndLoanApply,2022-05-14 00:56:59,EndLoanApply,5,1,0,0,1,...,0.0,0.0,17,0,4.657497e-16,0.000000e+00,0.0,0.0,0.000000e+00,8.5
4,20,2022-05-23,OpenApp,2022-05-23 01:33:32,start,5,13,10,1,2,...,0.0,0.0,8,0,5.349238e-16,0.000000e+00,2.5,0.0,0.000000e+00,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246280,879691,2022-03-14,Login,2022-03-14 09:05:24,OpenApp,3,10,10,0,0,...,0.0,0.0,78,0,1.000000e+00,0.000000e+00,2.5,0.0,0.000000e+00,39.0
246281,879692,2022-04-25,EndLoanApply,2022-04-25 14:00:41,StartLoanApply,4,8,0,8,0,...,6.0,0.0,36,0,-4.480707e-15,-4.000000e+00,3.0,0.0,-3.000000e+00,15.0
246282,879693,2022-05-20,EndLoanApply,2022-05-20 16:39:54,StartLoanApply,5,5,0,0,5,...,0.0,2.0,11,0,5.000000e-01,4.657497e-16,1.0,0.0,1.583361e-15,5.5
246283,879695,2022-05-27,EndLoanApply,2022-05-27 12:51:23,ClickApply,5,1,0,0,1,...,0.0,1.0,4,0,4.657497e-16,0.000000e+00,0.5,0.0,0.000000e+00,2.0


In [ ]:
# cls_df.to_csv('/content/drive/Shareddrives/빅콘테스트/데이터/군집화/cluster_df.csv', index = False)

# 군집 분석
**K - means 모델 사용** 
- k = 6

In [ ]:
# cls_df = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/데이터/군집화/cluster_df.csv')

In [ ]:
# 변수 선택
df_cluster = cls_df[['app_count','app_variation',
                 'GetCreditInfo', 'credit_variation',
                 'UsePrepayCalc', 'prepay_variation',
                 'dsr_variation', 'UseDSRCalc',
                 'UseLoanManage', 'loan_variation',
                'applied_cnt', 'applied_variation',
                 'recency', 'new']]

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#표준화
sc = StandardScaler()
df_scaled = sc.fit_transform(df_cluster)
pd.DataFrame(df_scaled) 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.579156,0.033862,-0.196865,0.172818,-0.081489,-0.664209,-0.014186,-0.077448,-0.409241,0.312495,-0.667461,-0.625218,-0.660945,-0.20453
1,-0.486206,0.515980,0.268761,-0.246333,3.932398,0.215889,5.558235,-0.077448,0.665541,-1.349044,0.213025,0.567117,0.582672,-0.20453
2,-0.300304,-0.086667,-0.289990,0.382394,-0.081489,-0.077477,-0.014186,-0.077448,-0.289821,0.312495,-0.080470,2.355621,2.347805,-0.20453
3,-0.579156,-0.327726,-0.476240,0.172818,-0.081489,-0.664209,-0.014186,-0.077448,-0.409241,0.312495,-0.667461,-0.347006,-0.380128,-0.20453
4,0.536251,-0.327726,-0.383115,0.172818,-0.081489,0.802621,-0.014186,-0.077448,-0.409241,0.312495,0.800016,-0.704707,-0.741178,-0.20453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246280,0.257400,-0.086667,-0.289990,0.172818,-0.081489,0.802621,-0.014186,-0.077448,-0.409241,0.312495,0.800016,2.077409,2.066988,-0.20453
246281,0.071498,-0.327726,0.268761,-1.503787,-0.081489,1.095987,-0.014186,-0.077448,0.307280,-1.349044,1.093511,0.169672,0.382088,-0.20453
246282,-0.207354,-0.207196,-0.383115,0.172818,-0.081489,-0.077477,-0.014186,-0.077448,-0.050980,0.312495,-0.080470,-0.585474,-0.620828,-0.20453
246283,-0.579156,-0.327726,-0.476240,0.172818,-0.081489,-0.370843,-0.014186,-0.077448,-0.409241,0.312495,-0.373965,-0.863685,-0.901645,-0.20453


In [ ]:
# 클러스터링
kmeans = KMeans(n_clusters=6, random_state=0)
clusters = kmeans.fit(df_scaled)

# 클러스터링 변수 
df_cluster['cluster'] = clusters.labels_
df_cluster.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,app_count,app_variation,GetCreditInfo,credit_variation,UsePrepayCalc,prepay_variation,dsr_variation,UseDSRCalc,UseLoanManage,loan_variation,applied_cnt,applied_variation,recency,new,cluster
0,1,1.5,3.0,1.583361e-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,10,0,2
1,2,3.5,8.0,-1.000000e+00,1.0,1.5,0.5,0.0,9.0,-3.0,3.0,20.0,41,0,0


KMeans 함수에서 random_state = 0으로 지정해 centroid의 초기값을 고정함. centroid 초기값을 고정하여도 군집 계산 과정에서 완벽하게 재현되지 않음. 군집 라벨링 및 군집 별 수치(평균, 중앙값)가 완벽하게 재현되지 않을 수 있지만 각 군집의 특성과 해석은 변함없음.

In [ ]:
# cluster를 기준으로 데이터 개수
df_cluster.groupby('cluster').count()

,app_count,app_variation,GetCreditInfo,credit_variation,UsePrepayCalc,prepay_variation,dsr_variation,UseDSRCalc,UseLoanManage,loan_variation,applied_cnt,applied_variation,recency,new
cluster,,,,,,,,,,,,,,
0,70829,70829,70829,70829,70829,70829,70829,70829,70829,70829,70829,70829,70829,70829
1,206,206,206,206,206,206,206,206,206,206,206,206,206,206
2,142193,142193,142193,142193,142193,142193,142193,142193,142193,142193,142193,142193,142193,142193
3,9185,9185,9185,9185,9185,9185,9185,9185,9185,9185,9185,9185,9185,9185
4,17032,17032,17032,17032,17032,17032,17032,17032,17032,17032,17032,17032,17032,17032
5,6840,6840,6840,6840,6840,6840,6840,6840,6840,6840,6840,6840,6840,6840


In [ ]:
# 그룹별 중앙값
df_cluster.groupby('cluster').median()

,app_count,app_variation,GetCreditInfo,credit_variation,UsePrepayCalc,prepay_variation,dsr_variation,UseDSRCalc,UseLoanManage,loan_variation,applied_cnt,applied_variation,recency,new
cluster,,,,,,,,,,,,,,
0,2.0,0.0,1.0,0.00,0.0,0.5,0.0,0.0,0.0,0.00,1.0,28.50,57.0,0.0
1,128.5,60.0,165.0,-19.25,0.0,2.0,0.0,0.0,121.5,-22.75,4.0,-0.25,0.0,0.0
2,4.0,0.5,3.0,0.00,0.0,0.5,0.0,0.0,1.0,0.00,1.0,5.50,11.0,0.0
3,6.0,1.0,4.0,0.00,0.0,1.0,0.0,0.0,2.0,0.00,2.0,6.50,14.0,1.0
4,17.0,1.0,8.0,-0.50,0.0,4.5,0.0,0.0,5.0,-0.50,9.0,2.00,6.0,0.0
5,34.0,11.0,35.0,-4.00,0.0,1.5,0.0,0.0,26.0,-5.00,3.0,0.50,2.0,0.0


In [ ]:
# 그룹별 평균값
df_cluster.groupby('cluster').mean()

,app_count,app_variation,GetCreditInfo,credit_variation,UsePrepayCalc,prepay_variation,dsr_variation,UseDSRCalc,UseLoanManage,loan_variation,applied_cnt,applied_variation,recency,new
cluster,,,,,,,,,,,,,,
0,2.838665,-0.736876,2.119454,-0.535254,0.007617,-0.001652,1.610023e-18,0.003457,1.072652,-0.294423,1.343376,-0.369290,54.577504,0.000000
1,36.768160,7.436300,37.298645,8.012702,0.087881,0.007245,5.645465e-03,0.033120,28.433384,6.109804,7.777192,1.963681,3.037636,0.063982
2,16.533591,-1.251351,18.844015,-2.565637,1.317375,-0.406178,-4.915058e-01,1.396139,16.721236,-2.363320,2.988417,-0.367181,26.504247,0.062548
3,24.937831,-3.548459,18.446175,-3.146129,0.054839,-0.016393,3.599241e-04,0.008704,13.958118,-2.737517,6.639880,-1.470126,13.895884,0.018062
4,5.964725,0.990540,4.007238,0.562931,0.009310,-0.000112,4.658684e-04,0.004551,2.553251,0.380322,2.102653,0.454183,10.730659,0.000000
5,8.778177,0.234552,5.846715,-0.028285,0.014143,-0.001414,6.527415e-04,0.006527,3.789708,0.034704,2.799391,0.019256,22.482158,1.000000


In [ ]:
# 모델 저장하기
import joblib

joblib.dump(kmeans, '/content/drive/Shareddrives/빅콘테스트/제출파일/Raw 데이터/clustering_model.pkl') # 불러올 때는 kmeans = joblib.load('파일명')

['/content/drive/Shareddrives/빅콘테스트/제출파일/Raw 데이터/clustering_model.pkl']

In [ ]:
import plotly.graph_objects as go

ks = range(1,25)
inertias = []

for k in ks:
  model = KMeans(n_clusters=k)
  model.fit(df_scaled)
  inertias.append(model.inertia_)

In [ ]:
fig = go.Figure(data = go.Scatter(x=list(ks), y = list(inertias), mode = 'lines+markers', ))
fig.update_layout(
    autosize=False,
    width=600)
fig.show()

In [ ]:
df_cluster.to_csv('/final_df_cluster.csv', index = False)

# html 파일로 변환

In [6]:
!jupyter nbconvert --to html "/content/drive/Shareddrives/빅콘테스트/제출파일/코드내용 정리된 파일(Ipynb, Py)/군집화_합본.ipynb"

[NbConvertApp] Converting notebook /content/drive/Shareddrives/빅콘테스트/제출파일/코드내용 정리된 파일(Ipynb, Py)/군집화_합본.ipynb to html
[NbConvertApp] Writing 463429 bytes to /content/drive/Shareddrives/빅콘테스트/제출파일/코드내용 정리된 파일(Ipynb, Py)/군집화_합본.html
